In [ ]:
%matplotlib widget
from bmcs_shell.api import WBTessellation4P, WBShellAnalysis, MATS2DElastic, FETriangularMesh, FETS2DMITC, MATSShellElastic, AbaqusLink 
import numpy as np

In [ ]:
tmodel = MATS2DElastic(E=28000, nu=0.2)
# tmodel = MATSShellElastic(E=28000, nu=0.2)
n_phi_plus=2
n_x_plus=1
wbm = WBShellAnalysis(h=10, tmodel=tmodel, id='2_cells_wb')

In [ ]:
data = dict(gamma=np.pi/2-1.24, a=500, a_high=3000, b=900, b_high=3000, c=400, c_high=2000,
            n_phi_plus=n_phi_plus, n_x_plus=n_x_plus, show_nodes=False)
wbm.geo.trait_set(**data)

In [ ]:
wbm.interact()

In [ ]:
# wbm.bcs.bc_loaded_array = []

In [ ]:
wbm.export_abaqus()

In [ ]:
wbm.run()

In [ ]:
wbm.xdomain.Eia

In [ ]:
wbm.xdomain.F_N

In [ ]:
wbm.xdomain.mesh.X_Id

In [ ]:
min(wbm.hist.U_t[-1])

In [ ]:
np.arange(60).reshape(-1, 5)

In [ ]:
[print('dof: ' + str(b.dof) + ', value: ' + str(b.value)) for b in wbm.bc]

In [ ]:
xx_Ei, yy_Ei = np.einsum('...a->a...', wbm.xdomain.x_Eia)
yy_Ei[:, 1] - yy_Ei[:, 2]

In [ ]:
wbm.xdomain.mesh.I_Fi

In [ ]:
bc_fixed = [BCDof(var= 'u', dof=dof, value=0) for dof in fixed_dofs]
bc_loaded = [BCDof(var= 'f', dof=dof, value=u_cntl) for dof in loaded_dofs]
ts = TStepBC(
    domains=[(xdomain, tmodel)],
    bc=bc_fixed + bc_loaded,
)

In [ ]:
wbm.bc

In [ ]:
wbm.xdomain.mesh.X_Id

In [ ]:
X_Fid = wbm.xdomain.mesh.X_Id[wbm.xdomain.mesh.I_Fi]
X_Fid

In [ ]:
wbm.xdomain.Fa_normals[0]

In [ ]:
wbm.xdomain.mesh.X_Id[wbm.xdomain.mesh.I_Fi][0][0]

## Understanding bc

In [ ]:
xdomain = wbm.xdomain

In [ ]:
fixed_xyz_nodes = xdomain.bc_J_xyz
fixed_xyz_nodes

In [ ]:
xdomain.I_CDij

In [ ]:
fixed_x_nodes = xdomain.bc_J_x
fixed_x_nodes

In [ ]:
np.hstack([fixed_xyz_nodes, fixed_x_nodes])

In [ ]:
fixed_nodes = np.unique(np.hstack([fixed_xyz_nodes, fixed_x_nodes]))
fixed_nodes

In [ ]:
fixed_xyz_nodes = xdomain.bc_J_xyz
fixed_x_nodes = xdomain.bc_J_x
fixed_nodes = np.unique(np.hstack([fixed_xyz_nodes, fixed_x_nodes]))
fixed_xyz_dofs = (fixed_xyz_nodes[:, np.newaxis] * 3 + np.arange(3)[np.newaxis, :]).flatten()
fixed_x_dofs = (fixed_x_nodes[:, np.newaxis] * 3).flatten()
fixed_dofs = np.unique(np.hstack([fixed_xyz_dofs, fixed_x_dofs]))



In [ ]:
xmodel = wbm.fe_domain[0].xmodel
xmodel.mesh.I_CDij

In [ ]:
_, idx_remap = xmodel.mesh.unique_node_map

In [ ]:
idx_remap[xmodel.mesh.I_CDij]

In [ ]:
idx_remap[xmodel.mesh.I_CDij[(0, -1),:,(0, -1),:].flatten()]

In [ ]:
xmodel

In [ ]:
import numpy as np
alpha_range = np.linspace(0.5, 1.3, 10)
a_range = np.linspace(300,500,10)

In [ ]:
tmodel = MATS2DElastic(E=28000, nu=0.2)
import numpy as np
U_grid = []
sig_grid = []
for alpha in alpha_range:
    U_list = []
    sig_list = []
    for a in a_range:
        wbm = WBShellAnalysis(F=-20000, h=10, tmodel=tmodel)
        data = dict(alpha=alpha, a=a, a_high=2000, b=1250, b_high=2000, c=400, c_high=2000,
                    n_phi_plus=n_phi_plus, n_x_plus=n_x_plus)
        wbm.wb_mesh.trait_set(**data)
        max_U = wbm.get_max_vals()
        
        U1 = wbm.hist.U_t[-1]
        eps1 = wbm.fe_domain[0].xmodel.map_U_to_field(U1)
        sig1, _ = wbm.fe_domain[0].tmodel.get_corr_pred(eps1,1)
        sig_x, sig_y, tau = sig1.T
        sig_ab = np.einsum('ab...->...ab', np.array([[sig_x, tau], [tau, sig_y]], dtype=np.float_))
        sig_val, sig_dir = np.linalg.eig(sig_ab)
        sig_list.append([np.min(sig_val), np.max(sig_val)])

        print(alpha, a, max_U)
        U_list.append(max_U)
    U_grid.append(U_list)
    sig_grid.append(sig_list)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
fig = plt.figure()
ax = fig.gca(projection='3d')
X, Y = np.meshgrid(alpha_range, a_range)
Z = np.array(U_grid)
surf = ax.plot_surface(X, Y, Z, 
                       color='green',
                       #linewidth=0, antialiased=False)
                    alpha=.0, linewidth=5,
                       #cmap=cm.gist_earth,antialiased=False, 
                shade=False,rstride=1, cstride=1)
Z = np.array(sig_grid)[:,:,1]
surf = ax.plot_surface(X, Y, Z,color='red', 
                       #cmap=cm.copper,
                    alpha=.3, linewidth=3,
                       #cmap=cm.gist_earth,antialiased=False, 
                shade=False,rstride=1, cstride=1)
Z = -np.array(sig_grid)[:,:,0]
surf = ax.plot_surface(X, Y, Z,color='blue', 
                       #cmap=cm.copper,
                    alpha=.6, linewidth=3,
                       #cmap=cm.gist_earth,antialiased=False, 
                shade=False,rstride=1, cstride=1)
ax.set_xlabel(r'$\alpha~[\mathrm{rad}]$')
ax.set_ylabel(r'$a~[\mathrm{mm}]$')
ax.set_zlabel(r'max compression [MPa]')


In [ ]:
i_alpha, i_a = 0, -1
wbm = WBShellAnalysis(F=-1000, tmodel=tmodel)
print('alpha', alpha_range[i_alpha],'a', a_range[i_a] )
data = dict(alpha=alpha_range[i_alpha], 
            a=a_range[i_a], a_high=2000, b=1250, b_high=2000, c=400, c_high=2000,
            n_phi_plus=n_phi_plus, n_x_plus=n_x_plus)
wbm.wb_mesh.trait_set(**data)
wbm.wb_mesh.interact()